In [1]:
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU,Dense
from keras.layers import ZeroPadding2D,Flatten
from keras.layers import UpSampling2D,Lambda
from keras.layers import add, Concatenate,GlobalAveragePooling2D,Softmax
from keras.models import Model
import tensorflow as tf
from keras.applications import vgg19
import tensorflow_datasets as tfds
import os
import cv2 
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import tensorflow_datasets as tfds

In [2]:
!mkdir .train
!mkdir .train/HR
!mkdir .train/LR

!mkdir .test
!mkdir .test/HR
!mkdir .test/LR
train,test = tfds.load('div2k/realistic_mild_x4',split=['train','validation'], shuffle_files=True,as_supervised=True)

2021-12-31 12:40:45.381269: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


EXTRACTING {'train_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_mild.zip', 'valid_lr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_mild.zip', 'train_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip', 'valid_hr_url': 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip'}


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Shuffling div2k-train.tfrecord...:   0%|          | 0/800 [00:00<?, ? examples/s]

Generating validation examples...:   0%|          | 0/100 [00:00<?, ? examples/s]

Shuffling div2k-validation.tfrecord...:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset div2k downloaded and prepared to /root/tensorflow_datasets/div2k/realistic_mild_x4/2.0.0. Subsequent calls will reuse this data.



User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_PATTERN='hyper,hyper'
   KMP_REDUCTION_BARRIER='1,1'
  

In [3]:
count=0
for  lr,hr in train:
    # Saving the hr image
    loc= '.train/HR/image_'+str(count)+'.png'
    cv2.imwrite(loc,hr.numpy())
    
     # Saving the hr image
    loc= '.train/LR/image_'+str(count)+'.png'
    cv2.imwrite(loc,lr.numpy())
    count+=1
    
count=0
for  lr,hr in test:
    # Saving the hr image
    loc= '.test/HR/image_'+str(count)+'.png'
    cv2.imwrite(loc,hr.numpy())
    
     # Saving the hr image
    loc= '.test/LR/image_'+str(count)+'.png'
    cv2.imwrite(loc,lr.numpy())
    count+=1
    
     
    

2021-12-31 12:44:29.220152: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [4]:
def augmentation(path_to_HR,path_to_LR):

    count=800 
    for i in range(800):

        if np.random.sample() >0.7:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])

            imgflip_HR= cv2.flip(img_HR,0) #flipping the image from x-axis
            imgflip_LR= cv2.flip(img_LR,0)
            #Getting the location of the images
           
            imgHR_loc= '.train/HR/'+'image_'+str(count) +'.png'
            imgLR_loc= '.train/LR/'+'image_'+str(count) +'.png'
    
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)


        if np.random.sample() >0.75:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])

            
            #Rotating the image at an angle of 90
            imgflip_HR= cv2.rotate(img_HR, cv2.ROTATE_90_CLOCKWISE)
            imgflip_LR= cv2.rotate(img_LR, cv2.ROTATE_90_CLOCKWISE)
            #Getting the location of the images

            imgHR_loc= '.train/HR/'+'image_'+str(count) +'.png'
            imgLR_loc= '.train/LR/'+'image_'+str(count) +'.png'

    
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)


        if np.random.sample() >0.75:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])

            
            #Rotating the image at an angle of 270
            imgflip_HR= cv2.rotate(img_HR, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            imgflip_LR= cv2.rotate(img_LR, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            
            #Getting the location of the images

            imgHR_loc= '.train/HR/'+'image_'+str(count) +'.png'
            imgLR_loc= '.train/LR/'+'image_'+str(count) +'.png'

            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)   




In [5]:
paths_to_HR_images= ['.train/HR/'+name for name in os.listdir('.train/HR')]
paths_to_LR_images= ['.train/LR/'+name for name in os.listdir('.train/LR')]

In [6]:
augmentation(paths_to_HR_images,paths_to_LR_images)
path_to_LR= os.listdir('.train/LR')
print('The total LR image size after augmentation',len(path_to_LR))
path_to_HR= os.listdir('.train/HR')
print('The toatl HR image size after augmentation',len(path_to_HR))

The total LR image size after augmentation 1479
The toatl HR image size after augmentation 1479


In [7]:
# # @tf.function
# def Total_patches(paths,patch_size,name):
    
#     total_data=[]
#     shapes_img=[]
#     count=1
#     for path in tqdm(paths):
         
#         patch,sh,count=getting_patches(path,patch_size,name,count)
#         # print(sh)
#         total_data.extend(patch)
#         shapes_img.extend(sh)
#     loc='/content/drive/MyDrive/SRGAN/5LR'+'.pkl'   
#     with open(loc,'wb') as f:
#          pickle.dump(shapes_img,f)

# #getting patches of the input images
# def getting_patches(paths,patch_size,name,count):

#     scale= 1 if name=='LR' else 2
#     patch_size=patch_size*scale
#     for i in range(1):
#         patches=[]
#         shapes=[]
#         image= cv2.imread(paths, cv2.IMREAD_UNCHANGED)
#         ny= image.shape[0]//patch_size
#         nx= image.shape[1]//patch_size
#         shapes.append([ny,nx])


#         for i in range(ny):

#           starty=i*patch_size

#           for j in range(nx):

#             start= j*patch_size
#             end= patch_size + j*patch_size

#             Lr_patch= image[starty:patch_size+starty,start:end]
#             # Lr_patch= image[:patch_size,start:en
#             # parent_dir="/content/drive/MyDrive/SRGAN/5LR/"
#             # directory=1
#             # if count%7500==0:
#             #   directory +=1
#             #   pathh = os.path.join(parent_dir, str(directory))
#             #   os.mkdir(pathh)

#             folder=1
#             if count%7500 == 1:
#               folder = + 1
#             location='/content/drive/MyDrive/SRGAN/5LR/'+str(folder)+'/'+str(count)+'.png'
#             # location= pathh +"/"+str(count)+'.png'

#             count=count+1
#             cv2.imwrite(location,Lr_patch)
#             patches.append(Lr_patch)


#     return patches,shapes,count


# #combining the patches
# def combine_patches(patches,shapes):
  
#       h,w= shapes
#       horizontal_image=[] 

#       for i in range(h):
#         image= np.hstack((patches[i*w],patches[i*w+1])) #combining first two patched images
#         index= i*w+2 #As the first two patched imaged have been combined, now index selects third image

#         for j in range(w-2): #-2 because the first the patches have been joined already
#             image=np.hstack((image,patches[index+j]))
         
#         horizontal_image.append(image) 
    
#       actual_image=np.vstack((horizontal_image[0],horizontal_image[1]))
#       for i in range(h-2):
#         actual_image= np.vstack((actual_image,horizontal_image[2+i]))

#       return actual_image    


# def Total_combine_patches(patches,shapes,data_size,image_index=None):

#   temporary= 0   
#   total_actual_image=[]
#   if image_index != None:
#       image_index=image_index-1
#       for i in range(image_index):
#           hs= shapes[i][0]          
#           ws= shapes[i][1]
#           temporary= temporary+hs*ws
       
#       h_,w_= shapes[image_index]
#       return  combine_patches(patches[temporary:temporary+h_*w_],shapes[image_index])   


#   for i in tqdm(range(data_size)):  
#       h,w= shapes[i]
#       image=combine_patches(patches[temporary:temporary+h*w],shapes[i])
#       total_actual_image.append(image)
#       temporary= h*w+temporary      

#   return total_actual_image
    
